In [7]:
# 调包和初始化
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
def hanming(a,b):
    res = 0
    for i,j in zip(a,b):
        res += (0 if i==j else 1)
    return res

In [36]:
# 导入训练数据
df_train = pd.read_csv("train.csv")
x = np.asarray(df_train.iloc[:,2:]).astype(np.float64)
y = np.array(df_train.iloc[:,1])
# 导入测试数据
df_test = pd.read_csv("test.csv")
x_pred = np.array(df_test.iloc[:,1:])
# 把训练数据分为训练集和验证集
x_train = x[:170000]
y_train = y[:170000]
x_test = x[170000:]
y_test = y[170000:]
# 归一化
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(x_train)  
x_train_scaler = scaler.transform(x_train)  
x_test_scaler = scaler.transform(x_test)
x_pred_scaler = scaler.transform(x_pred)

In [ ]:
# 随机森林 0.843
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train,y_train)
print(clf.score(x_test,y_test))  # 0.843
pd.DataFrame({'win':clf.predict(x_pred)}).to_csv('submission_randomforest.csv')

In [ ]:
# 极限随机树 0.8389
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier()
clf.fit(x_train,y_train)
print(clf.score(x_test,y_test))  # 0.8389
pd.DataFrame({'win':clf.predict(x_pred)}).to_csv('submission_extratrees.csv')

In [ ]:
# AdaBoost 0.8277
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(x_train,y_train)
print(clf.score(x_test,y_test))  # 0.8277
pd.DataFrame({'win':clf.predict(x_pred)}).to_csv('submission_adaboost.csv')

In [ ]:
# 梯度树提升 0.8391
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf.fit(x_train,y_train)
print(clf.score(x_test,y_test))  # 0.8391
pd.DataFrame({'win':clf.predict(x_pred)}).to_csv('submission_gradientboost.csv')

In [ ]:
# 多层感知机 0.8472
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=(15))
clf.fit(x_train_scaler,y_train)
print(clf.score(x_test_scaler,y_test))    # 0.8472
pd.DataFrame({'win':clf.predict(x_pred_scaler)}).to_csv('submission_mlp.csv')

In [37]:
data = []
name = ['randomforest','extratrees','gradientboost','adaboost','mlp']
for x in name:
    data.append([x,pd.read_csv("submission/submission_"+x+".csv")['win']])

In [38]:
for i in range(len(data)):
    for j in range(i+1,len(data)):
        print(data[i][0],data[j][0],hanming(data[i][1],data[j][1]))

randomforest extratrees 1135
randomforest gradientboost 1333
randomforest adaboost 2132
randomforest mlp 1562
extratrees gradientboost 1440
extratrees adaboost 2157
extratrees mlp 1675
gradientboost adaboost 1773
gradientboost mlp 1649
adaboost mlp 2168


In [53]:
# 分类器投票表决
y_pred = []
for i in range(len(data[0][1])):
    sum = 0
    for j in range(len(data)):
        sum += data[j][1][i]
    y_pred.append(1 if sum >= len(data)/2 else 0)
pd.DataFrame({'win':y_pred}).to_csv("submission.csv")